<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkDeltaMerge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install delta-spark==2.0.0

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkDeltaMerge") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.4-py2.py3-none-any.whl size=282040920 sha256=32a72c223cc2a78aecf6187d907a22b39522d04f48ad3cabe7984d0f29189ed0
  Stored in directory: /root/.cache/pip/wheels/81/6c/a6/af459ed3106da8f1171a265c2982425fe390aa418a47d2a8b4
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-conn

In [2]:
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
df=spark.read.format('csv').option('header','true').option('inferSchema',True).load('/content/drive/MyDrive/Abc/Ola_data/products.csv')

In [6]:
df.write.format('delta').mode('overwrite').saveAsTable('Products_table')

In [7]:
df.createOrReplaceTempView('Products_temp')

In [9]:
spark.sql('select * from products_temp').show(4)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
|    Wired Earbuds|Electronics Hub|   PROD101| STORE01|25.99|California|    USA|
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 4 rows



In [12]:
spark.sql('create or replace table Products (product string,store string,product_id string,MRP int,\
            state string, country string)using delta location "/content/sample_data/store"')

DataFrame[]

In [14]:
spark.sql('select * from Products').show()

+-------+-----+----------+---+-----+-------+
|product|store|product_id|MRP|state|country|
+-------+-----+----------+---+-----+-------+
+-------+-----+----------+---+-----+-------+



In [17]:
spark.sql(
    "Merge into Products as target using Products_temp as source on target.product_id = source.product_id\
     when matched then update set target.product=source.product,\
        target.store=source.store,\
        target.product_id=source.product_id,\
        target.MRP=source.MRP,\
        target.state=source.state,\
        target.country=source.country\
     when not matched then insert * "
)

DataFrame[]

In [19]:
spark.sql('select * from Products').show(5)

+-------------+------------+----------+----+--------+-------+
|      product|       store|product_id| MRP|   state|country|
+-------------+------------+----------+----+--------+-------+
|Foot Massager|Home Gadgets|   PROD175| 100| Florida|    USA|
| Dehumidifier|Office Depot|   PROD162| 150|Illinois|    USA|
|   Hand Mixer|Office Depot|   PROD154|  25|Illinois|    USA|
|  Smart Scale| Mobile Mart|   PROD179|null|   Texas|    USA|
| Portable SSD|   PC Palace|   PROD129|null|New York|    USA|
+-------------+------------+----------+----+--------+-------+
only showing top 5 rows



**END OF CODE**